**Autor** : Hader Azzini

**e-mail**: hader.azzini@gmail.com

#  <a name="resumo"> RESUMO </a>

Este notebook tem o objetivo ser avaliação de um case para o processe seletivo do Zé Delivery. O [case](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting) é relativo a previsão de vendas de 45 lojas do walmart

#  <a name="indice">  Índice </a>

* [Resumo](#resumo)
* [Índice](#indice)
* [Seção 1 - "Organizando a Mesa de Trabalho" : Instalações, Imports, Unzip e Leitura dos Dados](#secao_1)
* [Seção 2 - Explorando os Dados](#secao_2)
* [Seção 3 - Modelagem Usando o Fbprophet](#secao_3)
* [Seção 4 - Modelando todos Departamentos](#secao_4)
* [Seção 5 - Previsões para o Teste](#secao_5)
* [Conclusão](#conclusao)

#  <a name="secao_1"> Seção 1 - "Organizando a Mesa de Trabalho" : Imports, Unzip e Leitura dos Dados</a>
[Voltar ao índice](#indice)

### Instalações de pacotes úteis

In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet
!pip install scikit-learn==0.24.2

### Imports

In [ ]:
import os
from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

from zipfile import ZipFile, is_zipfile
import shutil

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from prophet import Prophet

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

from tqdm import tqdm

### Organização dos Dados

Prepação dos Arquivos brutos (Unzip e cópia para uma nova pasta). 

Isso facilita o uso e conserva a base original segura

In [ ]:
KAGGLE_DATA_PATH = '../input/walmart-recruiting-store-sales-forecasting/'

listando_arquivos_do_diretorio = lambda mypath:[f for f in listdir(mypath) if isfile(join(mypath, f))]

all_kaggle_files = listando_arquivos_do_diretorio(KAGGLE_DATA_PATH)
all_kaggle_files

In [ ]:
RAW_DATA_PATH = './raw_data/'

if not os.path.exists(RAW_DATA_PATH):
    os.makedirs(RAW_DATA_PATH)

for filename in all_kaggle_files:
    file_path =  join(KAGGLE_DATA_PATH,filename)
    
    if is_zipfile(file_path):
        print("Extraindo...{}".format(filename))
        with ZipFile(file_path) as f:
            f.extractall(path=RAW_DATA_PATH)
    else:
        print("Copiando...{}".format(filename))
        shutil.copyfile(file_path, join(RAW_DATA_PATH,filename))
        
print('\nOrganização dos arquivos brutos concluida!')

Verificando visualmente se todos os arquivos estão corrtamente no diretório

In [ ]:
all_raw_files = listando_arquivos_do_diretorio(RAW_DATA_PATH)
all_raw_files

### Leitura dos Dados

A Leitura dos dados para um dicionário facilita a manipulação, uma vez que a base é pequena em tamanho, mas tem diferentes arquivos

In [ ]:
data_inputs = {}

for filename in all_raw_files:
    file_path = join(RAW_DATA_PATH,filename)
    try:
        name = filename.split('.')[0]
        data_inputs[name] =  pd.read_csv(file_path)
        print('Arquivo {} carregado!'.format(filename))
    except Exception as e:
        print('Falha ao ler o arquivo %s. Reason: %s' % (file_path, e))

Verificando visualmente uma amostra dos arquivos

In [ ]:
data_inputs['features'].head()

In [ ]:
data_inputs['train'].head()

In [ ]:
data_inputs['stores'].head()

In [ ]:
data_inputs['sampleSubmission'].head()

In [ ]:
data_inputs['test'].head()

#  <a name="secao_2"> Seção 2  - Explorando os Dados</a>
[Voltar ao índice](#indice)

### Manipulando os Dados
Manipulando os dados para unir features e vendas com o departamento da loja

In [ ]:
def unindo_vendas_e_features(df_features,df_vendas,loja,departamento,merge_cols =['Date','Store','IsHoliday']):
    
    df_vendas_departamento= df_vendas.query('Store == {} & Dept== {}'.format(loja,departamento))
    df_features_loja = df_features.query('Store == {}'.format(loja))
    
    df_loja_departamento = pd.merge(df_features_loja, df_vendas_departamento, on=merge_cols)
    
    return df_loja_departamento

def unir_vendas_features_por_departamento(data_inputs,loja,departamento):
    return unindo_vendas_e_features(data_inputs['features'],data_inputs['train'],loja=loja,departamento=departamento)

In [ ]:
df_1_1 = unir_vendas_features_por_departamento(data_inputs,loja=1,departamento=1)

In [ ]:
df_1_1

### Análise Exploratória
Vamos usar o **Pandas Profiling** para exploração e avaliação da qualidade dos dados. O pandas profiling acelar a exploração de dados e permite a geração rapida de insights. Inicialmente vamos fazer a exploração para um departamento de uma loja, nesse caso o Departamento 1 da Loja 1.

In [ ]:
profile = ProfileReport(df_1_1, title="Pandas Profiling Report")
profile.to_notebook_iframe()

Para o Departamento 1 da Loja 1 podemos econtrar pela analise exploratória que **a temperatura tem correlação negativa com as vendas semanais**. Isso indica que este departamento vai melhor durante a estação fria. Entretanto, não podemos afirmar com certeza que é a temperatura que causa a melhora nas vendas. Se o departamento vendesse itens de inverno (casacos, luvas, patins para o gelo...) essa poderia ser uma possibilidade, entretanto, pode ser apenas uma questão de concidência com os feriados e festas de fim de ano, onde o comércio tende aumentar mesmo em países com verão ao fim do ano.

A ausência de correlação com os Markdowns sugere que esse departamento não é muito afetado por eles, entretanto, como os Markdowns foram registrados somente a partir de novembro de 2011, é necessário avalir a melhor somente neste período.

Vamos analisar melhor os graficos de temperatura e vendas semanais para entender a relação entre estas features.

In [ ]:
def gerando_figuras_dois_eixos(df,x_eixo,y1_eixo,y2_eixo):

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df[x_eixo], y=df[y1_eixo], name=y1_eixo),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df[x_eixo], y=df[y2_eixo], name=y2_eixo),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="{} and {}".format(y1_eixo,y2_eixo)
    )

    # Set x-axis title
    fig.update_xaxes(title_text=x_eixo)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>{}</b>".format(y1_eixo), secondary_y=False)
    fig.update_yaxes(title_text="<b>{}</b>".format(y2_eixo), secondary_y=True)

    fig.show()


In [ ]:
gerando_figuras_dois_eixos(df_1_1,'Date','Weekly_Sales','Temperature')

Os picos de vendas nos feriados, especialmente o feriado de abril, onde a temperatura já está em torno de 70ºF, sugere que a temperatura de fato não é a causa do aumento de vendas

Agora vamos analisar melhor a influência dos Markdowns:

In [ ]:
gerando_figuras_dois_eixos(df_1_1,'Date','Weekly_Sales','MarkDown1')

Aparentemente os Mardkowns do início de 2012 tem correlação com as vendas semanais, entreanto parece que isso não se reflete no meio do ano.

Vamos descartar o periodo sem Mardowns para verificar a influencia deles. Como os valores faltantes estão como NaN podemos fazer um dropna, para ter apenas linhas que tenham valores para os Markdowns.

In [ ]:
df_1_1_with_markdown = df_1_1.dropna().reset_index(drop=True)

Vamos fazer nova análise exploratórioa para o periodo com registros de Markdown

In [ ]:
profile = ProfileReport(df_1_1_with_markdown, title="Pandas Profiling Report")
profile.to_notebook_iframe()

Fazendo nova exploração de dados deste período encontramos que **Weekly_Sales tem correlação alta com a Date, Unemployment,CPI,Markdown2 e Markdown5**. Perceba que algumas dessas correlações não vem da tradicional correlação de Pearson mas sim da Phik. Correlações de Phik tem interpretação mais sutil uma vez que capturam dependencias não lineares, assim não podemos ser categóricos em afirmar relações de causa/consequência. 

Como descrito na documentação sobre Phik no Pandas Profiling:

>"Phik (φk) is a new and practical correlation coefficient that works consistently between categorical, ordinal and interval variables, captures non-linear dependency and reverts to the Pearson correlation coefficient in case of a bivariate normal input distribution."


Entretanto, em geral as correlações Phik indicam um bom palpite para investigações mais detalhadas. Assim sendo vamos verificar os gráficos no tempo dessas correlações

In [ ]:
gerando_figuras_dois_eixos(df_1_1_with_markdown,'Date','Weekly_Sales','MarkDown5')

MarkDown5 tem um pico em torno do natal mas mantem valores relativaments estáveis ao longo do ano, se assemelhando com as vendas. Entretanto, de maio a a setembro as vendas não apresentam muita variação. Isso sugere que o MarkDown5 ajuda em algumas ocasiões mas não é uma bala de prata quando se pensa em aumentar as vendas

In [ ]:
gerando_figuras_dois_eixos(df_1_1_with_markdown,'Date','Weekly_Sales','MarkDown2')

MarkDown2 tem pico no fim do ano mas se mantem estavel na maior parte dele, onde as vendas também não mostram variações

In [ ]:
gerando_figuras_dois_eixos(df_1_1_with_markdown,'Date','Weekly_Sales','Unemployment')

Com a diminuição do desemprego ao longo de 2012, o final do ano pode ter tido um aumento em relacão a 2011, uma vez que se tem uma parcela maior da população com pagamentos em suas mãos

In [ ]:
gerando_figuras_dois_eixos(df_1_1_with_markdown,'Date','Weekly_Sales','CPI')

O Aumento da inflação ao longo de 2012 pode ter retraido as vendas absolutas no fim de 2012, uma vez que o consumidor não será capaz de comprar a mesma quantidades de intens com a mesma quantia que usou em 2011

### Influência dos Feriados

Ainda é preciso entender melhor o impacto que os feriados tem na venda para isso vamos verificar qual a discrepância da variabilidade das vendas entre semanas que tem feriados e as que não tem.

In [ ]:
def figura_boxplot_influencia_feriados(data_inputs,loja,departamentos):

    fig = go.Figure()
    
    for dept in departamentos:
        fig = verificando_influencia_dos_feriados(fig,data_inputs,loja=loja,departamento=dept)
        
    fig.show()
    
    
def verificando_influencia_dos_feriados(fig,data_inputs,loja,departamento):

    df = unir_vendas_features_por_departamento(data_inputs,loja=loja,departamento=departamento)

    
    fig.add_trace(go.Box(y=df.query('IsHoliday == False')['Weekly_Sales'],name='Sem Feriado - Dept {}'.format(departamento)))
    fig.add_trace(go.Box(y=df.query('IsHoliday == True')['Weekly_Sales'],name='Feriado - Dept {}'.format(departamento) ))

    fig.update_layout(
            title_text="Loja {}".format(loja)
        )

    return fig

In [ ]:
departamentos = [a+1 for a in range(4)]
figura_boxplot_influencia_feriados(data_inputs,loja=1,departamentos=departamentos)
figura_boxplot_influencia_feriados(data_inputs,loja=2,departamentos=departamentos)
figura_boxplot_influencia_feriados(data_inputs,loja=3,departamentos=departamentos)

As analises dos 3 primeiros departamentos das 3 primeiras lojas indica que a **variabilidade das vendas em semanas de feriados para o Departamento 1 é maior do que para os Departamentos 2 e 3**. Isso pode ser percebido pela ditância do q3 (75% das amostras) e q1 (25% das amostras). Isso indica que os departamentos 2 e 3 provalvelmente não são tão influenciados por feriados.

Poderíamos focar em diversas analises exploratórias aqui de cada departamento de cada loja. Mas isso seria custoso e se afastaria do objetivo de usar aprendizado de máquina e ao invés do aprendizado humano. Assim, uma vez que temos um entedimento incial dos dados fornecido pela análise exploratória de parte da amostra, vamos focar daqui para frente na modelagem, de forma a obter modelos mais amplos para todos os departamentos.

#  <a name="secao_3"> Seção 3 -  Modelagem Usando o Fbprophet</a>
[Voltar ao índice](#indice)

A partir da constatação que alguns departamentos são influenciados nas vendas por feriados é interessante testar uma modelagem capaz de levar os feriados em consideração. Como o [Fbprophet](https://juanitorduz.github.io/fb_prophet/) tem essa funcionalidade ele é uma boa opção inicial. Vamos usar apenas a série temporal de vendas nas primeiras análises uma vez que as correlações de Pearson com outras variaveis não tiveram valor em módulo maior que 0.7. Caso o modelo não se mostre com resultados aceitaveis podemos investigar uma análise mulitvariável.

In [ ]:
#O Prophet para fazer consumir os dados da série temporal deve ter a coluna de data nomeada como 'ds' e a coluna de registros como 'y'
df = df_1_1[['Date','Weekly_Sales']]
df.columns =['ds','y']
df.head()

Vamos obter as semanas dos feriados e configura para o formato esperado pelo Prophet

In [ ]:
feriados = pd.DataFrame({
  'holiday': 'holiday',
  'ds': data_inputs['train'].query("Store==1 & IsHoliday == True").reset_index(drop=True)['Date'],
  'lower_window': 0,
  'upper_window': 1,
})

In [ ]:
feriados.head()

In [ ]:
# A classe é apenas para silenciar o fb prophet, naturalmente ele gera muitos wargnings
# from https://stackoverflow.com/questions/11130156/suppress-stdout-stderr-print-from-python-functions
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

def modelando_com_prophet(df,df_future,holidays=None):
    
    if holidays is not None:
        m = Prophet(holidays=holidays)
    else:
        m = Prophet()
        
    with suppress_stdout_stderr():
        m.fit(df)
        
    forecast = m.predict(df_future)
    
    return m,forecast

def gerando_grafico_de_validacao(df_train,df_test,forecast,titulo=None):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_train['ds'], y=df_train['y'], name='Treino'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Previsão'))
    
    if not df_test['y'].isna().all():
        fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y'], name='Realizado'))
    
    if titulo is not None:
        fig.update_layout(title=titulo)
    
    fig.show()

Vamos mostrar visualmente como o prophet leva em consideração os feriados

In [ ]:
test_size=3*4 # 3 meses
i=1 # ultimos 3 meses

df_train, df_test =df.query(" ds  <= '2011-09-02' "), df.query(" ds  > '2011-09-02' & ds <= '2012-10-26' ")

m,forecast=  modelando_com_prophet(df_train,df_test)
gerando_grafico_de_validacao(df_train,df_test,forecast,titulo='Sem Considerar os Feriados')


m,forecast=  modelando_com_prophet(df_train,df_test,holidays=feriados)
gerando_grafico_de_validacao(df_train,df_test,forecast,titulo="Considerando os Feriados")

Como a amostra de treino é pequena para o exemplo a previsão foi ruim, praticamente uma linha reta. Mas quando consideramos os feriados vemos pequenos picos na previsão coerentes com os picos que aconteceram nas vendas

Para vericarmos se o modelo faz a previsão adequadamente vamos dividir a [série temporal em cinco pedaços para validação](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py).

Como métricas vamos usar [MAPE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html) e [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html). Quanto mais próximas de zero melhor é o modelo. Durante as análises nos focaremos mais na MAPE pois ela permite uma comparação relativa entre departamentos e lojas

In [ ]:
test_size=3*4 # 3 meses

for i in range(5, 0, -1):
    
    df_train, df_test = df.iloc[:-i*test_size], df.iloc[-i*test_size:]
    
    mensagem_treino = 'Treino :{} a {}'.format(df_train.iloc[0]['ds'],df_train.iloc[-1]['ds'])
    mensagem_teste = 'Teste :{} a {}'.format(df_test.iloc[0]['ds'],df_test.iloc[-1]['ds'])
    
    m,forecast=  modelando_com_prophet(df_train,df_test,holidays=feriados)
    
    gerando_grafico_de_validacao(df_train,df_test,forecast, titulo =' e '.join([mensagem_treino,mensagem_teste]))
    
    print('\n')
    print(' MAPE: {}'.format(mean_absolute_percentage_error(df_test['y'].values,forecast['yhat'].values)))
    print(' MAE: {}'.format(mean_absolute_error(df_test['y'].values,forecast['yhat'].values)))
    
    print('---')

Como pode ser visto a medida que os dados de teino aumentam o valor de MAPE diminui. O que é concistente com o fato de termos uma amostra limitada e por isso temos poucos anos para que o modelo absorva a sazonalidade apresentada ao longo do ano. 

Outro aspecto é que nas melhores condições o modelo consegue gerar MAPE = 0.087, valor bem adequado.

#  <a name="secao_4"> Seção 4  - Modelando todos Departamentos</a>
[Voltar ao índice](#indice)

Agora vamos efeturar a modelagem para cada um dos departamentos para cada uma das lojas. Iniciando por modelar todos os departamentos da Loja

### Todos os deparamentos da Loja 1

In [ ]:
def preparando_dados_para_prophet(data_inputs,loja=1,departamento=1):

    df_train = data_inputs['train'].query('Store =={} & Dept =={}'.format(loja,departamento))[['Date','Weekly_Sales']]
    df_train.columns=['ds','y']


    df_test = data_inputs['test'].query('Store =={} & Dept =={}'.format(loja,departamento))[['Date']]
    df_test.columns=['ds']

    return df_train,df_test


def modelando_com_validacao_simples(df,test_size=3*4,verbose=True):
    
    resultados = {}
    
    df_train, df_test = df.iloc[:-test_size], df.iloc[-test_size:]

   
    resultados['m'],resultados['forecast']=  modelando_com_prophet(df_train,df_test)

    
    resultados['MAPE'] = mean_absolute_percentage_error(df_test['y'].values, resultados['forecast']['yhat'].values)
    resultados['MAE'] = mean_absolute_error(df_test['y'].values,resultados['forecast']['yhat'].values)
    
    if verbose:
        print('Treino :{} ... {}'.format(df_train.iloc[0]['ds'],df_train.iloc[-1]['ds']))
        print('Teste :{} ... {}'.format(df_test.iloc[0]['ds'],df_test.iloc[-1]['ds']))
        print('MAPE :{}'.format(resultados['MAPE']))
        print('MAE :{}'.format(resultados['MAE']))
    
    return resultados

def modelando_apenas_final(df,test_size=3*4):
    
    resultados = {}
    
    df_train, df_test = df.iloc[:-test_size], df.iloc[-test_size:]

    print('Treino :{} ... {}'.format(df_train.iloc[0]['ds'],df_train.iloc[-1]['ds']))
    print('Teste :{} ... {}'.format(df_test.iloc[0]['ds'],df_test.iloc[-1]['ds']))
    resultados['m'],resultados['forecast']=  modelando_com_prophet(df_train,df_test)

    
    resultados['MAPE'] = mean_absolute_percentage_error(df_test['y'].values, resultados['forecast']['yhat'].values)
    resultados['MAE'] = mean_absolute_error(df_test['y'].values,resultados['forecast']['yhat'].values)
    
    
    print('MAPE :{}'.format(resultados['MAPE']))
    print('MAE :{}'.format(resultados['MAE']))
    
    return resultados

df_train,df_test = preparando_dados_para_prophet(data_inputs,loja=2,departamento=1)
resultados = modelando_com_validacao_simples(df_train,test_size=3*4)

In [ ]:
def modelando_validando_por_loja(data_inputs,loja,test_size=3*4,verbose=False):

    resultdos_por_dept = {}
    departamentos = data_inputs['train'].query('Store == {}'.format(loja))['Dept'].unique()

    for dept in tqdm(departamentos):
        if verbose:
            print('----- Dept {} ------'.format(dept))
        try:
            df_train,df_test = preparando_dados_para_prophet(data_inputs,loja=loja,departamento=dept)
            resultdos_por_dept[dept] = modelando_com_validacao_simples(df_train,test_size=test_size,verbose=verbose)
        except:
            print('\n ERRO no Dept {} da Loja {}!\n'.format(dept,loja))
            
    return resultdos_por_dept

def formatando_resultado(resultdos_por_dept):
    
    metricas = [[a,resultdos_por_dept[a]['MAPE'],resultdos_por_dept[a]['MAE']] for a in resultdos_por_dept]

    df_metricas = pd.DataFrame(metricas,columns=['Dept','MAPE','MAE'])
    
    return df_metricas

In [ ]:
loja=1

reusltados = modelando_validando_por_loja(data_inputs,loja,test_size=3*4)

df_metricas = formatando_resultado(reusltados)

In [ ]:
df_metricas = formatando_resultado(reusltados)
df_metricas

In [ ]:
df_metricas.describe()

In [ ]:
df_metricas.median()

Ao analisarmos os valores gerados pelo describe podemos ver que apenas 75 deparamentos foram processados.Para 2 (Depts 77 e 78) o algoritmo falhou e os demais números de departamento não existem para essa loja.

Outro ponto importante é que o MAPE atinge mínimo de 3.932695e-02 e máximo de 1.101713e+17, isso nos leva a verificar se o máximo é apenas um valor isolado ou o modelo tem resultado ruim. O valor da média foi puxado para cima por valor tão grande. Ao calcularmos a mediana constatamos que de forma geral a modelagem vai bem com MAPE de 0.121220.

Vamos retirar todos os outliers de MAPE (MAPE > 1)

In [ ]:
df_metricas_com_outliers = df_metricas[df_metricas['MAPE'] > 1 ]
df_metricas_sem_outliers = df_metricas[df_metricas['MAPE'] <= 1 ]

In [ ]:
df_metricas_com_outliers

Um total de 7 departamentos tem valores acima de 1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y= df_metricas_sem_outliers['MAPE'],name='loja 1'))
fig.show()

Para os deparamentos sem outliers vemos que 75% das amostras tem MAPE até 0.1773, ou seja boa parte dos departamentos tem modelos adequados. Na Figura abaixo temos o valor de MAPE por departamento

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_metricas_sem_outliers['Dept'],y=df_metricas_sem_outliers['MAPE']))
fig.update_layout(title='MAPE por Departamento da Loja 1',
                 xaxis_title="Departamento",
                 yaxis_title="MAPE")
fig.show()

### Todos Departamentos de Todas Lojas

**<span style="color:red">ATENÇÃO! a execução do código a seguir leva cerca de 1h 45minutos</span>** . Para agilizar o analise é possível saltar para o [processamento das métricas](#process_metricas). As células foram deixadas no tipo raw para evitar execução acidental

### <a name="process_metricas"> Processamento das métricas  </a> 

No caso de ter sido feito o salto para essa parte do código faça a obtenção do [arquivo](https://drive.google.com/file/d/1ahjQxiO9rMlBKbGSpRq6J3kwwWBywZyY/view?usp=sharing) depois via  **Add data > Upload a dataset** faça o upload do arquivo **metricas_todas_loja_e_departamentos.csv**, nomeando o dataset como **metricaslojas** para que a proxima linha funcione adequadamente. Ou então coloque o nome que prefir e depois copie o path do arquivo e cole abaixo

In [ ]:
df_metricas = pd.read_csv("../input/metricaslojaswalmart/metricas_todas_loja_e_departamentos.csv",index_col='Dept')

In [ ]:
df_metricas

Retirando os outlies (MAPE > 1 )

In [ ]:
aux = df_metricas.copy()

for a in aux.columns:
    aux[a] = np.where(aux[a] > 1, np.nan, aux[a])

In [ ]:
fig = px.imshow(aux.T)
fig.show()

Podemos ver que a cor roxa predominante mostra que o para a maioria das lojas e departamentos o modelo teve MAPE de 0 a 0.3. Além disso, certos departamentos apresentam regularidade de bons resultados de previsão para as 45 lojas, como o departamentos 2,4,8,13,40,46,90,95 e 97. Alguns departamentos tem reusltuados bem ruim para todas quase todas as lojas como 36,45 e 58

In [ ]:
fig = go.Figure()

for loja in aux.columns:
    fig.add_trace(go.Box(y=aux[loja],name=loja))

fig.update_layout(title='MAPE para cada Loja',
                 yaxis_title='MAPE',
                 xaxis_title='Loja')
fig.show()

O gráfico de Boxplot mostra que as medianas do MAPE para as 45 lojas estão abaixo de 0.2 indicando bom resultado para a maioria dos departamentos. Algumas lojas tem grande variabilidade de resultados como a 30,33,36,38 e 45

Como a maioria dos resultado está boa e devido ao tempo limitado para a tarefa não iremos proceder com a previsão usando multivariável, mas ela poderia ser investigada como outliers para os departamentos com MAPE elevado. Além disso poderia ser feita uma avaliação uma investigação mais detalhas porque certas departamentos não foram possíveis de modelar ou apresentaram outliers

#  <a name="secao_5"> Seção 5 - Previsões no formato do arquivo de Teste </a>
[Voltar ao índice](#indice)

Antes de executarmos a previsão conforme o arquivo de teste exemplo, para as datas indicadas, é importante treinar o modelo com todos os dados disponiveis, uma vez que já foi feita a validação

In [ ]:
def modelando_por_loja(data_inputs,loja,verbose=False):

    resultdos_por_dept = {}
    
    df_train = data_inputs['train'].query('Store == {}'.format(loja))
    df_test = data_inputs['test'].query('Store == {}'.format(loja))
    
    departamentos = df_train['Dept'].unique()
    
    for dept in tqdm(departamentos):
        if verbose:
            print('----- Dept {} ------'.format(dept))
        try:
            resultdos_por_dept[dept] = modelando_sem_validacao(df_train.query('Dept == {}'.format(dept)),df_test.query('Dept == {}'.format(dept)))
        except:
            print('\n ERRO no Dept {} da Loja {}!\n'.format(dept,loja))
            
    return resultdos_por_dept

def modelando_sem_validacao(df_train,df_test):
    
    train,test = preparando_dfs_para_prophet(df_train,df_test)
    m,forecast = modelando_com_prophet(train,test,holidays=feriados)
    
    return {'m':m,'forecast':forecast}


def preparando_dfs_para_prophet(train,test):

    df_train = train[['Date','Weekly_Sales']]
    df_train.columns=['ds','y']


    df_test = test[['Date']]
    df_test.columns=['ds']

    return df_train,df_test

In [ ]:
#import pickle

resultados_por_loja = {}
for loja in data_inputs['train']['Store'].unique()[:3]:
    print('--- Loja {} ---'.format(loja))
    resultados_por_dept = modelando_por_loja(data_inputs,loja)
    resultados_por_loja[loja] = resultados_por_dept
    
    #with open('loja_{}.pickle'.format(loja), 'wb') as handle:
    #    pickle.dump(resultados_por_dept, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
loja = 1
departamento = 1

df_train = data_inputs['train'].query('Store == {}'.format(loja))
df_test = data_inputs['test'].query('Store == {}'.format(loja))

test = df_test.query('Dept == {}'.format(departamento))[['Date']]
test.columns = ['ds']
test['y']=np.NaN


train = df_train.query('Dept == {}'.format(departamento))[['Date','Weekly_Sales']]
train.columns = ['ds','y']
gerando_grafico_de_validacao(train,test,resultados_por_loja[loja][departamento]['forecast'])

Para o Departamento 1 da Loja 1 a previsão feita pelo modelo do Prophet visualmente parece ser coerente com o comportamento dos últimos anos, embora as vendas para o periodo previsto apresentem picos menores.

In [ ]:
def criando_id(data,loja,departamento):    
    return "{}_{}_".format(loja,departamento)+data.strftime("%Y-%m-%d")


previsoes = []

for loja in resultados_por_loja:
    for departamento in resultados_por_loja[loja]:
        df_predict = resultados_por_loja[loja][departamento]['forecast'][['ds','yhat']]
        df_predict.columns = ['Id','Weekly_Sales']
        
        previsoes.append(pd.concat([df_predict['Id'].apply(criando_id,args=(loja,departamento)),df_predict['Weekly_Sales']],axis=1))
        
df_sample_submmit = pd.concat(previsoes).reset_index(drop=True)

In [ ]:
df_sample_submmit

In [ ]:
df_sample_submmit.to_csv('result_to_submmit.csv')

#  <a name="conclusao"> Conclusão </a>
[Voltar ao índice](#indice)

Inicialmente neste case preparamos todas as bibliotecas e arquivos de entrada. Em seguida passamos para a análise exploratória, sendo os dados unidos em um dataframe único e aplicado o Pandas Profiling para acelerar a análise exploratória. Para o Departamento 1 da Loja 1 encontramos pela analise exploratória que a temperatura tem correlação negativa com as vendas semanais. Isso indica que este departamento vai melhor durante a estação fria. Entretanto, não podemos afirmar com certeza que é a temperatura a causa da melhora nas vendas. Em feriados e festas de fim de ano o comércio tende a aumentar mesmo em países com verão em tal período.

Ao ser feita uma nova análise exploratória para o período no qual existem registros de Markdown encontramos que Weekly_Sales tem correlação alta com a Date, Unemployment, CPI, Markdown2 e Markdown5. Entretanto algumas dessas correlações não vem da tradicional correlação de Pearson mas sim da Phik. Correlações de Phik tem interpretação mais sutil uma vez que capturam dependências não lineares, assim não podemos ser categóricos em afirmar relações de causa/consequência. Em geral as correlações Phik indicam um bom palpite para investigações mais detalhadas. Assim sendo foram verificados os gráficos no tempo dessa variáveis com correlação, mas não é possível fazer afirmativas categóricas sobre causa e consequência.

Em seguida investigou-se melhor o impacto que os feriados têm nas vendas. As análises dos 3 primeiros departamentos das 3 primeiras lojas indicam que a variabilidade das vendas em semanas de feriados para o Departamento 1 é maior do que para os Departamentos 2 e 3. Além disso os departamentos 2 e 3 apresentam sinais de serem influenciados por feriados.
A partir da constatação que alguns departamentos são influenciados nas vendas por feriados foi testada uma modelagem capaz de levar os feriados em consideração. Como o FB Prophet tem essa funcionalidade ele foi a opção inicial. Foi usada apenas a série temporal de vendas semanais uma vez que as correlações de Pearson com outras variáveis não tiveram valor em módulo maior que 0.7. Caso o modelo não tivesse mostrado resultados aceitáveis poderia ser investigado uma modelagem multivariável, mas não foi necessário esse aumento no custo computacional.

Para verificar se o modelo faz a previsão adequadamente a série temporal foi dividida em cinco pedaços para validação. Como métricas foram usadas MAPE e MAE, mas durante as análises o foco foi na MAPE pois ela permite uma comparação relativa entre departamentos e lojas. Como foi visto à medida que os dados de treino aumentam o valor de MAPE diminui. O que é consistente com o fato de termos uma amostra limitada e por isso temos poucos anos para que o modelo absorva a sazonalidade apresentada ao longo do ano. Outro aspecto é que nas melhores condições o modelo consegue gerar MAPE = 0.087, valor bem adequado.

Ao analisarmos as métricas descritivas para as departamentos das Loja 1 podemos ver que apenas 75 departamentos foram processados. Para dois departamentos (Depts 77 e 78) o algoritmo falhou e os demais números de departamento não existem para essa loja. Outro ponto importante é que o MAPE atinge mínimo de 3.932695e-02 e máximo de 1.101713e+17, isso nos leva a verificar se o máximo é apenas um valor isolado ou o modelo tem resultado ruim. Ao calcularmos a mediana constatamos que de forma geral a modelagem vai bem com mediana de MAPE igual a 0.121220.

Para os departamentos sem outliers vemos que 75% das amostras tem MAPE até 0.1773, ou seja boa parte dos departamentos tem modelos adequados. 
Para ser feita a modelagem com validação para as 45 lojas o código levou cerca de 1h 45minutos.  
O mapa de calor e mostrou visualmente que o para a maioria das lojas e departamentos o modelo teve MAPE entre 0 a 0.3. Além disso, certos departamentos apresentam regularidade de bons resultados de previsão para as 45 lojas, como os departamentos 2,4,8,13,40,46,90,95 e 97. Entretanto alguns departamentos tem resultados ruins para quase todas as lojas como 36,45 e 58. O gráfico de Boxplot mostra que as medianas do MAPE para as 45 lojas estão abaixo de 0.2 indicando bom resultado para a maioria dos departamentos. Algumas lojas tem grande variabilidade de resultados como a 30,33,36,38 e 45.

Como a maioria dos resultados está adequada e devido ao tempo limitado para a tarefa não foi feita uma previsão usando multivariável, mas ela poderia ser usada investigada os outliers de departamentos com MAPE elevado. Além disso poderia ser feita uma investigação mais detalhas porque certos departamentos não foram possíveis de modelar ou apresentaram outliers.
Por fim, os dados de treino foram integralmente usados, foi feito a previsão para as datas de teste e gerado um arquivo no formato para submissão
